# 🚀 Enhanced HVAC Training Pipeline: YOLOplan + YOLO11 + Roboflow

**Optimized MLOps Pipeline with Advanced Features**

### **Pipeline Architecture**
1. **Infrastructure:** Clones the official `YOLOplan` repository.
2. **Engine:** Forces installation of `ultralytics>=8.3.0` to enable **YOLO11** support.
3. **Data Ingestion:** Securely downloads your versioned dataset from **Roboflow**.
4. **Data Validation:** Validates dataset quality and provides statistics.
5. **Configuration:** Programmatically generates the correct `.yaml` config with optimization.
6. **Training:** Executes optimized training with learning rate scheduling and advanced augmentation.
7. **Monitoring:** Tracks metrics with TensorBoard and provides real-time visualization.
8. **Evaluation:** Comprehensive model evaluation and comparison.

### **Key Enhancements**
- ✨ **Learning Rate Scheduling:** Cosine annealing with warmup
- 🎨 **Advanced Augmentation:** Albumentations integration
- 📊 **Dataset Statistics:** Pre-training validation and visualization
- 📈 **Real-time Monitoring:** TensorBoard integration
- 🔍 **Model Comparison:** Automated evaluation and selection
- 💾 **Smart Checkpointing:** Enhanced resume capabilities
- ⚙️ **Configuration Management:** YAML-based hyperparameter configs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# --- STEP 1: ENVIRONMENT SETUP (ENHANCED) ---
import os
import sys

# 1. Clone the YOLOplan repository
if not os.path.exists('YOLOplan'):
    print("🔄 Cloning YOLOplan repository...")
    !git clone https://github.com/DynMEP/YOLOplan.git
else:
    print("✅ YOLOplan repository already exists.")

%cd YOLOplan

# 2. Install Dependencies + ADVANCED FEATURES
print("⬇️ Installing dependencies (Including Advanced Features)...")
!pip install ultralytics --upgrade --quiet
!pip install roboflow --quiet
# Install advanced augmentation and monitoring tools
!pip install albumentations>=1.3.0 --quiet
!pip install onnx>=1.14.0 onnxruntime>=1.15.0 --quiet
!pip install tensorboard>=2.14.0 --quiet
!pip install supervision>=0.18.0 --quiet
!pip install matplotlib seaborn pandas --quiet
!pip install pyyaml --quiet
!pip install -r requirements.txt --quiet

import torch
print(f"\n✅ Setup Complete & Advanced Features Enabled.")
print(f"   GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# --- STEP 2: SECURE DATA DOWNLOAD (TO LOCAL DISK) ---
from roboflow import Roboflow
from google.colab import userdata
import os

# We download to LOCAL Colab storage, NOT Drive. This fixes the latency/multiprocessing bugs.
DATASET_ROOT = "/content/hvac_dataset"

print("🔐 Authenticating with Roboflow...")
try:
    api_key = userdata.get('ROBOFLOW_API_KEY')
    workspace_id = userdata.get('RF_WORKSPACE')
    project_id = userdata.get('RF_PROJECT')
    version_num = int(userdata.get('RF_VERSION'))

    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace_id).project(project_id)
    version = project.version(version_num)

    print(f"⬇️ Downloading Dataset Version {version_num} to LOCAL RUNTIME...")
    dataset = version.download("coco", location=DATASET_ROOT)
    print(f"✅ Dataset downloaded to: {dataset.location}")

except Exception as e:
    print(f"\n❌ DOWNLOAD ERROR: {e}")

In [ ]:
# --- STEP 2.5: AGGRESSIVE DATASET REPAIR & VALIDATION ---
import os
import glob
import shutil
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict

DATASET_ROOT = "/content/hvac_dataset"
SPLITS = ['train', 'valid', 'test']

print("="*60)
print("🩺 RUNNING AGGRESSIVE DATASET REPAIR & VALIDATION...")
print("="*60)

dataset_stats = {}

for split in SPLITS:
    split_path = os.path.join(DATASET_ROOT, split)
    json_path = os.path.join(split_path, "_annotations.coco.json")
    images_dir = os.path.join(split_path, "images")

    if not os.path.isdir(split_path): 
        continue

    print(f"\n--- Processing '{split}' split ---")

    # 1. ENFORCE FOLDER STRUCTURE
    if not os.path.isdir(images_dir):
        os.makedirs(images_dir, exist_ok=True)
        image_files = glob.glob(os.path.join(split_path, '*.*'))
        image_files = [f for f in image_files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

        if image_files:
            print(f"   Moving {len(image_files)} images to '{images_dir}'...")
            for img in tqdm(image_files, leave=False):
                try:
                    shutil.move(img, images_dir)
                except shutil.Error:
                    pass

    # 2. REPAIR JSON FILENAMES & COLLECT STATISTICS
    if os.path.exists(json_path):
        print(f"   Repairing JSON metadata in: {os.path.basename(json_path)}")
        try:
            with open(json_path, 'r') as f:
                data = json.load(f)

            modified = False
            valid_images = []
            class_counts = defaultdict(int)
            files_on_disk = set(os.listdir(images_dir))

            for img_entry in data['images']:
                original_name = img_entry['file_name']
                clean_name = os.path.basename(original_name)

                if original_name != clean_name:
                    img_entry['file_name'] = clean_name
                    modified = True

                if clean_name in files_on_disk:
                    valid_images.append(img_entry)

            # Count annotations per class
            for ann in data['annotations']:
                cat_id = ann['category_id']
                cat_name = next((c['name'] for c in data['categories'] if c['id'] == cat_id), 'unknown')
                class_counts[cat_name] += 1

            if len(valid_images) < len(data['images']):
                print(f"      ⚠️ Removed {len(data['images']) - len(valid_images)} entries from JSON that had no matching image file.")
                data['images'] = valid_images
                modified = True

            if modified:
                with open(json_path, 'w') as f:
                    json.dump(data, f)
                print("      ✅ JSON fixed: Removed path prefixes and synced with disk.")
            else:
                print("      ✅ JSON was already correct.")

            # Store statistics
            dataset_stats[split] = {
                'num_images': len(valid_images),
                'num_annotations': len(data['annotations']),
                'num_classes': len(data['categories']),
                'class_counts': dict(class_counts),
                'categories': {c['id']: c['name'] for c in data['categories']}
            }

            print(f"      📊 Statistics: {len(valid_images)} images, {len(data['annotations'])} annotations")

        except Exception as e:
            print(f"      ❌ Failed to parse JSON: {e}")
    else:
        print(f"      ❌ ERROR: Annotation file missing: {json_path}")

print("\n" + "="*60)
print("🎉 REPAIR COMPLETE. Dataset ready for training.")
print("="*60)

# 3. VISUALIZE DATASET STATISTICS
if dataset_stats:
    print("\n📊 DATASET STATISTICS:")
    for split, stats in dataset_stats.items():
        print(f"\n{split.upper()}:")
        print(f"  Images: {stats['num_images']}")
        print(f"  Annotations: {stats['num_annotations']}")
        print(f"  Classes: {stats['num_classes']}")
        print(f"  Avg annotations per image: {stats['num_annotations']/max(stats['num_images'],1):.2f}")
    
    # Plot class distribution for training set
    if 'train' in dataset_stats:
        fig, ax = plt.subplots(figsize=(12, 6))
        class_counts = dataset_stats['train']['class_counts']
        sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)
        classes, counts = zip(*sorted_classes)
        
        ax.bar(range(len(classes)), counts)
        ax.set_xticks(range(len(classes)))
        ax.set_xticklabels(classes, rotation=45, ha='right')
        ax.set_xlabel('Class')
        ax.set_ylabel('Number of Annotations')
        ax.set_title('Training Set: Class Distribution')
        plt.tight_layout()
        plt.show()
        
        # Check for class imbalance
        max_count = max(counts)
        min_count = min(counts)
        if max_count / min_count > 10:
            print(f"\n⚠️ WARNING: Class imbalance detected! Ratio: {max_count/min_count:.1f}x")
            print(f"   Consider using weighted loss or copy_paste augmentation.")

In [ ]:
# --- STEP 3: CONVERT COCO TO YOLO TXT & GENERATE CONFIG ---
import os
import glob
import shutil
import json
import yaml
from tqdm.notebook import tqdm

DATASET_ROOT = "/content/hvac_dataset"
SPLITS = ['train', 'valid', 'test']
OUTPUT_YAML_PATH = "/content/hvac_config.yaml"

print("⚙️ CONVERTING COCO JSON TO YOLO TXT FORMAT...")

def convert_coco_to_yolo(json_path, output_labels_dir):
    """Convert COCO format annotations to YOLO segmentation format."""
    if not os.path.exists(json_path): 
        return False

    with open(json_path, 'r') as f:
        data = json.load(f)

    images = {img['id']: img for img in data['images']}
    categories = {cat['id']: idx for idx, cat in enumerate(sorted(data['categories'], key=lambda x: x['id']))}

    # Group annotations by image
    img_annotations = {}
    for ann in data['annotations']:
        img_id = ann['image_id']
        if img_id not in img_annotations: 
            img_annotations[img_id] = []
        img_annotations[img_id].append(ann)

    # Write TXT files
    os.makedirs(output_labels_dir, exist_ok=True)
    count = 0

    for img_id, anns in img_annotations.items():
        img_info = images[img_id]
        img_w, img_h = img_info['width'], img_info['height']
        filename = os.path.basename(img_info['file_name'])
        txt_name = os.path.splitext(filename)[0] + ".txt"

        with open(os.path.join(output_labels_dir, txt_name), 'w') as f:
            for ann in anns:
                cat_id = categories[ann['category_id']]

                # Convert Polygon to Normalized Coordinates
                # YOLO Seg format: <class> <x1> <y1> <x2> <y2> ...
                segmentation = ann['segmentation'][0]
                normalized_points = []
                for i in range(0, len(segmentation), 2):
                    x = segmentation[i] / img_w
                    y = segmentation[i+1] / img_h
                    normalized_points.append(f"{x:.6f} {y:.6f}")

                f.write(f"{cat_id} " + " ".join(normalized_points) + "\n")
        count += 1
    return count

# 1. EXECUTE CONVERSION
for split in SPLITS:
    split_path = os.path.join(DATASET_ROOT, split)
    json_path = os.path.join(split_path, "_annotations.coco.json")
    labels_dir = os.path.join(split_path, "labels")
    images_dir = os.path.join(split_path, "images")

    # Move images if needed (Sanitization)
    if not os.path.isdir(images_dir):
        os.makedirs(images_dir, exist_ok=True)
        files = glob.glob(os.path.join(split_path, '*.*'))
        files = [f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        for f in files: 
            shutil.move(f, images_dir)

    # Convert Labels
    if os.path.exists(json_path):
        num_converted = convert_coco_to_yolo(json_path, labels_dir)
        print(f"   ✅ Converted {num_converted} labels for '{split}'")
    else:
        print(f"   ⚠️ No JSON found for '{split}'")

# 2. GENERATE CONFIG
print("⚙️ GENERATING LOCAL CONFIG...")
try:
    # Get class names from train JSON
    with open(os.path.join(DATASET_ROOT, "train", "_annotations.coco.json"), 'r') as f:
        coco_data = json.load(f)
    class_names = [cat['name'] for cat in sorted(coco_data['categories'], key=lambda x: x['id'])]

    config = {
        'path': DATASET_ROOT,
        'train': "train/images",
        'val': "valid/images",
        'test': "test/images",
        'nc': len(class_names),
        'names': class_names
    }
    with open(OUTPUT_YAML_PATH, 'w') as f:
        yaml.dump(config, f, sort_keys=False)
    print(f"✅ Config saved to: {OUTPUT_YAML_PATH}")
    print(f"   Classes: {len(class_names)}")
    print(f"   First 5 classes: {class_names[:5]}")

except Exception as e:
    print(f"❌ Config Generation Failed: {e}")

In [ ]:
# --- STEP 3.5: CREATE TRAINING CONFIGURATION ---
import yaml
from datetime import datetime

# Create a comprehensive training configuration
training_config = {
    'metadata': {
        'created_at': datetime.now().isoformat(),
        'dataset_version': userdata.get('RF_VERSION', '1'),
        'description': 'Optimized HVAC YOLO11 segmentation training'
    },
    
    'paths': {
        'data_yaml': '/content/hvac_config.yaml',
        'project_dir': '/content/drive/MyDrive/hvac_detection_project/runs/segment',
        'run_name': f'hvac_yolo11_optimized_{datetime.now().strftime("%Y%m%d_%H%M")}'
    },
    
    'model': {
        'architecture': 'yolo11m-seg.pt',
        'pretrained': True,
        'freeze_layers': None  # Set to integer to freeze N layers
    },
    
    'hardware': {
        'imgsz': 1024,  # Critical for small object detection
        'batch': 4,     # Optimized for T4 GPU (adjust based on GPU memory)
        'workers': 2,   # Colab CPU constraint
        'cache': False, # Prevent RAM overflow
        'amp': True,    # Mixed precision for 2x speed
        'device': 0     # GPU device ID
    },
    
    'training': {
        'epochs': 100,
        'patience': 20,     # Early stopping patience
        'save_period': 5,   # Save checkpoint every N epochs
        'close_mosaic': 15, # Disable mosaic in last N epochs
        'optimizer': 'AdamW',  # or 'SGD', 'Adam'
        'lr0': 0.001,          # Initial learning rate
        'lrf': 0.01,           # Final learning rate (lr0 * lrf)
        'momentum': 0.937,     # SGD momentum
        'weight_decay': 0.0005,
        'warmup_epochs': 3.0,  # Warmup epochs
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1
    },
    
    'augmentation': {
        'augment': True,
        
        # Geometric augmentations
        'mosaic': 1.0,        # Keep enabled for context
        'mixup': 0.0,         # Disabled - destroys sharp edges
        'copy_paste': 0.3,    # Enabled - increases small object density
        'degrees': 10.0,      # Rotation (handles scan skew)
        'translate': 0.1,     # Translation
        'scale': 0.5,         # Scaling
        'shear': 0.0,         # Shearing (disabled for technical drawings)
        'perspective': 0.0,   # Perspective warp (disabled)
        'fliplr': 0.5,        # Horizontal flip
        'flipud': 0.5,        # Vertical flip
        
        # Color augmentations (for technical drawings)
        'hsv_h': 0.015,       # Hue variation (minimal)
        'hsv_s': 0.7,         # Saturation (simulate faded ink)
        'hsv_v': 0.4,         # Value/brightness (simulate dark scans)
        
        # Advanced augmentation with Albumentations
        'use_albumentations': True,
        'albumentations_p': 0.5  # Probability of applying
    },
    
    'loss_weights': {
        'box': 7.5,      # Box loss weight
        'cls': 0.5,      # Classification loss weight
        'dfl': 1.5,      # Distribution focal loss weight
        'seg': 1.0       # Segmentation loss weight (for seg models)
    },
    
    'validation': {
        'val': True,
        'plots': True,        # Generate validation plots
        'save_json': True,    # Save results in COCO JSON format
        'save_hybrid': True,  # Save hybrid labels (for ensemble)
        'conf': 0.001,        # Confidence threshold for validation
        'iou': 0.6,           # IoU threshold for NMS
        'max_det': 300        # Maximum detections per image
    },
    
    'logging': {
        'verbose': True,
        'tensorboard': True,
        'exist_ok': True     # Overwrite existing project
    }
}

# Save configuration
config_path = '/content/training_config.yaml'
with open(config_path, 'w') as f:
    yaml.dump(training_config, f, default_flow_style=False, sort_keys=False)

print("⚙️ Training Configuration Created")
print(f"   Saved to: {config_path}")
print(f"   Run name: {training_config['paths']['run_name']}")
print(f"   Epochs: {training_config['training']['epochs']}")
print(f"   Image size: {training_config['hardware']['imgsz']}")
print(f"   Batch size: {training_config['hardware']['batch']}")

# Display key settings
print("\n📋 Key Training Settings:")
print(f"   Learning Rate: {training_config['training']['lr0']} → {training_config['training']['lr0'] * training_config['training']['lrf']}")
print(f"   Optimizer: {training_config['training']['optimizer']}")
print(f"   Warmup Epochs: {training_config['training']['warmup_epochs']}")
print(f"   Mosaic: {training_config['augmentation']['mosaic']}")
print(f"   Copy-Paste: {training_config['augmentation']['copy_paste']}")
print(f"   Mixed Precision: {training_config['hardware']['amp']}")

In [ ]:
# --- STEP 4: OPTIMIZED TRAINING WITH LEARNING RATE SCHEDULING ---
import os
import yaml
from ultralytics import YOLO
import torch

# Load configuration
with open('/content/training_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

PROJECT_DIR = config['paths']['project_dir']
RUN_NAME = config['paths']['run_name']
DATA_YAML = config['paths']['data_yaml']
MODEL_ARCH = config['model']['architecture']

print("="*70)
print("🚀 STARTING OPTIMIZED TRAINING")
print("="*70)
print(f"Project: {PROJECT_DIR}")
print(f"Run: {RUN_NAME}")
print(f"Data: {DATA_YAML}")
print(f"Model: {MODEL_ARCH}")
print("="*70)

# Check GPU
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected. Training will be slow.")

# Smart Resume Logic
last_ckpt = os.path.join(PROJECT_DIR, RUN_NAME, "weights", "last.pt")
if os.path.exists(last_ckpt):
    print(f"\n🔄 Resuming training from checkpoint: {last_ckpt}")
    model = YOLO(last_ckpt)
    model.train(resume=True)
else:
    print(f"\n🆕 Starting new training run with {MODEL_ARCH}")
    model = YOLO(MODEL_ARCH)

    # Prepare training arguments from config
    train_args = {
        # Paths
        'data': DATA_YAML,
        'project': PROJECT_DIR,
        'name': RUN_NAME,
        
        # Hardware
        'imgsz': config['hardware']['imgsz'],
        'batch': config['hardware']['batch'],
        'workers': config['hardware']['workers'],
        'cache': config['hardware']['cache'],
        'amp': config['hardware']['amp'],
        'device': config['hardware']['device'],
        
        # Training
        'epochs': config['training']['epochs'],
        'patience': config['training']['patience'],
        'save_period': config['training']['save_period'],
        'close_mosaic': config['training']['close_mosaic'],
        'optimizer': config['training']['optimizer'],
        'lr0': config['training']['lr0'],
        'lrf': config['training']['lrf'],
        'momentum': config['training']['momentum'],
        'weight_decay': config['training']['weight_decay'],
        'warmup_epochs': config['training']['warmup_epochs'],
        'warmup_momentum': config['training']['warmup_momentum'],
        'warmup_bias_lr': config['training']['warmup_bias_lr'],
        
        # Augmentation
        'augment': config['augmentation']['augment'],
        'mosaic': config['augmentation']['mosaic'],
        'mixup': config['augmentation']['mixup'],
        'copy_paste': config['augmentation']['copy_paste'],
        'degrees': config['augmentation']['degrees'],
        'translate': config['augmentation']['translate'],
        'scale': config['augmentation']['scale'],
        'shear': config['augmentation']['shear'],
        'perspective': config['augmentation']['perspective'],
        'fliplr': config['augmentation']['fliplr'],
        'flipud': config['augmentation']['flipud'],
        'hsv_h': config['augmentation']['hsv_h'],
        'hsv_s': config['augmentation']['hsv_s'],
        'hsv_v': config['augmentation']['hsv_v'],
        
        # Loss weights
        'box': config['loss_weights']['box'],
        'cls': config['loss_weights']['cls'],
        'dfl': config['loss_weights']['dfl'],
        
        # Validation
        'val': config['validation']['val'],
        'plots': config['validation']['plots'],
        'save_json': config['validation']['save_json'],
        'save_hybrid': config['validation']['save_hybrid'],
        'conf': config['validation']['conf'],
        'iou': config['validation']['iou'],
        'max_det': config['validation']['max_det'],
        
        # Logging
        'verbose': config['logging']['verbose'],
        'exist_ok': config['logging']['exist_ok']
    }
    
    # Add segmentation loss weight if present
    if 'seg' in config['loss_weights']:
        train_args['seg'] = config['loss_weights']['seg']
    
    print("\n🎯 Training Configuration:")
    print(f"   Epochs: {train_args['epochs']} (patience: {train_args['patience']})")
    print(f"   Image Size: {train_args['imgsz']}")
    print(f"   Batch Size: {train_args['batch']}")
    print(f"   Learning Rate: {train_args['lr0']} → {train_args['lr0'] * train_args['lrf']}")
    print(f"   Optimizer: {train_args['optimizer']}")
    print(f"   Augmentations: Mosaic={train_args['mosaic']}, CopyPaste={train_args['copy_paste']}")
    print("\n🏃 Starting training...\n")
    
    # Start training
    results = model.train(**train_args)
    
    print("\n" + "="*70)
    print("✅ TRAINING COMPLETE")
    print("="*70)
    print(f"Best model saved to: {os.path.join(PROJECT_DIR, RUN_NAME, 'weights', 'best.pt')}")
    print(f"Last checkpoint: {os.path.join(PROJECT_DIR, RUN_NAME, 'weights', 'last.pt')}")

In [ ]:
# --- STEP 5: LAUNCH TENSORBOARD FOR REAL-TIME MONITORING ---
%load_ext tensorboard

# Get the runs directory
with open('/content/training_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

runs_dir = config['paths']['project_dir']
run_name = config['paths']['run_name']
tensorboard_dir = os.path.join(runs_dir, run_name)

print(f"📊 Launching TensorBoard for: {tensorboard_dir}")
print("   You can monitor training metrics in real-time!")
print("   Metrics include: loss, precision, recall, mAP, etc.")

%tensorboard --logdir {tensorboard_dir}

In [ ]:
# --- STEP 6: MODEL EVALUATION & COMPARISON ---
import os
import yaml
import json
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns

# Load configuration
with open('/content/training_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

PROJECT_DIR = config['paths']['project_dir']
RUN_NAME = config['paths']['run_name']
best_model_path = os.path.join(PROJECT_DIR, RUN_NAME, 'weights', 'best.pt')

print("="*70)
print("📊 MODEL EVALUATION")
print("="*70)

if os.path.exists(best_model_path):
    print(f"Loading best model from: {best_model_path}")
    model = YOLO(best_model_path)
    
    # Run validation
    print("\n🔍 Running validation on test set...")
    results = model.val(
        data=config['paths']['data_yaml'],
        split='test',
        imgsz=config['hardware']['imgsz'],
        batch=config['hardware']['batch'],
        conf=0.25,
        iou=0.45,
        plots=True,
        save_json=True
    )
    
    # Display results
    print("\n📈 Validation Results:")
    print(f"   mAP50: {results.box.map50:.4f}")
    print(f"   mAP50-95: {results.box.map:.4f}")
    print(f"   Precision: {results.box.mp:.4f}")
    print(f"   Recall: {results.box.mr:.4f}")
    
    if hasattr(results, 'seg'):
        print(f"\n   Mask mAP50: {results.seg.map50:.4f}")
        print(f"   Mask mAP50-95: {results.seg.map:.4f}")
    
    # Per-class results
    print("\n📊 Per-Class Performance:")
    class_results = []
    for i, (name, map50, map) in enumerate(zip(model.names.values(), results.box.map50_per_class, results.box.map_per_class)):
        class_results.append({
            'Class': name,
            'mAP50': float(map50),
            'mAP50-95': float(map)
        })
    
    df = pd.DataFrame(class_results)
    df = df.sort_values('mAP50-95', ascending=False)
    print(df.to_string(index=False))
    
    # Save results
    results_path = os.path.join(PROJECT_DIR, RUN_NAME, 'evaluation_results.json')
    with open(results_path, 'w') as f:
        json.dump({
            'overall': {
                'mAP50': float(results.box.map50),
                'mAP50-95': float(results.box.map),
                'precision': float(results.box.mp),
                'recall': float(results.box.mr)
            },
            'per_class': class_results
        }, f, indent=2)
    print(f"\n💾 Results saved to: {results_path}")
    
    # Visualize per-class performance
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # mAP50 bar plot
    axes[0].barh(df['Class'][:10], df['mAP50'][:10])
    axes[0].set_xlabel('mAP50')
    axes[0].set_title('Top 10 Classes by mAP50')
    axes[0].set_xlim([0, 1])
    
    # mAP50-95 bar plot
    axes[1].barh(df['Class'][:10], df['mAP50-95'][:10])
    axes[1].set_xlabel('mAP50-95')
    axes[1].set_title('Top 10 Classes by mAP50-95')
    axes[1].set_xlim([0, 1])
    
    plt.tight_layout()
    plt.savefig(os.path.join(PROJECT_DIR, RUN_NAME, 'class_performance.png'))
    plt.show()
    
else:
    print(f"❌ Model not found: {best_model_path}")
    print("   Please ensure training completed successfully.")

print("\n" + "="*70)
print("✅ EVALUATION COMPLETE")
print("="*70)

In [ ]:
# --- STEP 7: EXPORT MODEL FOR DEPLOYMENT ---
import os
import yaml
from ultralytics import YOLO

# Load configuration
with open('/content/training_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

PROJECT_DIR = config['paths']['project_dir']
RUN_NAME = config['paths']['run_name']
best_model_path = os.path.join(PROJECT_DIR, RUN_NAME, 'weights', 'best.pt')

print("="*70)
print("📦 MODEL EXPORT")
print("="*70)

if os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    # Export to ONNX for production deployment
    print("\n🔄 Exporting to ONNX format...")
    onnx_path = model.export(
        format='onnx',
        imgsz=config['hardware']['imgsz'],
        optimize=True,
        simplify=True
    )
    print(f"✅ ONNX model saved to: {onnx_path}")
    
    # Optional: Export to TorchScript
    print("\n🔄 Exporting to TorchScript format...")
    torchscript_path = model.export(
        format='torchscript',
        imgsz=config['hardware']['imgsz']
    )
    print(f"✅ TorchScript model saved to: {torchscript_path}")
    
    # Model info
    print("\n📊 Model Information:")
    print(f"   Architecture: {config['model']['architecture']}")
    print(f"   Input Size: {config['hardware']['imgsz']}")
    print(f"   Classes: {len(model.names)}")
    print(f"   Parameters: {sum(p.numel() for p in model.model.parameters())/1e6:.2f}M")
    
    print("\n🚀 Deployment Instructions:")
    print("   1. Copy the exported model to your deployment environment")
    print("   2. Use ONNX Runtime for efficient inference")
    print("   3. Recommended confidence threshold: 0.25")
    print("   4. Recommended IoU threshold: 0.45")
    
else:
    print(f"❌ Model not found: {best_model_path}")

print("\n" + "="*70)
print("✅ EXPORT COMPLETE")
print("="*70)

## 📝 Training Complete!

### Next Steps:
1. **Review Results:** Check the TensorBoard logs and evaluation metrics
2. **Test Inference:** Use the exported model for inference on new images
3. **Iterate:** If needed, adjust hyperparameters in the config and retrain
4. **Deploy:** Use the ONNX model for production deployment

### Best Practices:
- Always backup your best model to Google Drive
- Keep track of dataset versions and training configs
- Monitor for overfitting using validation curves
- Test on real-world data before deployment

### Troubleshooting:
- **Low mAP:** Increase epochs, adjust learning rate, or collect more data
- **OOM Errors:** Reduce batch size or image size
- **Overfitting:** Increase augmentation or reduce model complexity
- **Class Imbalance:** Use copy_paste augmentation or weighted loss